## Original Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], 1)
application_df

/Users/leahikenberry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
#print(application_df.nunique())
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
app_counts = application_df.APPLICATION_TYPE.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_counts[app_counts < 528].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df.CLASSIFICATION.value_counts()
class_counts.head(10)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: CLASSIFICATION, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 1883].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/Users/leahikenberry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)

application_df.head()

/Users/leahikenberry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

/Users/leahikenberry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


2022-02-24 12:44:29.739918: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5708 - accuracy: 0.7212
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5559 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7323
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5497 - accuracy: 0.7336
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7359
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7348
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7357
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7348
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5454 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5351 - accuracy: 0.7412
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7406
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7417
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5351 - accuracy: 0.7409
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5350 - accuracy: 0.7404
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5348 - accuracy: 0.7416
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5349 - accuracy: 0.7408
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5347 - accuracy: 0.7410
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7412
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5341 - accuracy: 

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5620 - accuracy: 0.7238 - 210ms/epoch - 783us/step
Loss: 0.5619958639144897, Accuracy: 0.7238484025001526


### Optimization 1: Adding Another Hidden Layer

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 10)                310       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Adding checkpoints
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [21]:
# Saving checkpoints
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804
    )

In [22]:
# Compiling model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5728 - accuracy: 0.7215
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7293
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7327
Epoch 5/100
788/804 [============================>.] - ETA: 0s - loss: 0.5475 - accuracy: 0.7340
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7330
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7353
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7332
Epoch 9/100
804/804 [=============================

804/804 [==============================] - 1s 1ms/step - loss: 0.5358 - accuracy: 0.7408
Epoch 65/100
783/804 [============================>.] - ETA: 0s - loss: 0.5359 - accuracy: 0.7401
Epoch 00065: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5355 - accuracy: 0.7403
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5352 - accuracy: 0.7410
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5356 - accuracy: 0.7404
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5354 - accuracy: 0.7411
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5353 - accuracy: 0.7407
Epoch 70/100
777/804 [===========================>..] - ETA: 0s - loss: 0.5351 - accuracy: 0.7411
Epoch 00070: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5355 - accuracy: 0.7405
Epoch 71/100
80

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5695 - accuracy: 0.7257 - 215ms/epoch - 802us/step
Loss: 0.5694645047187805, Accuracy: 0.7257142663002014


### Optimzation 2: Adding Neurons

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 200)               8800      
                                                                 
 dense_8 (Dense)             (None, 100)               20100     
                                                                 
 dense_9 (Dense)             (None, 50)                5050      
                                                                 
 dense_10 (Dense)            (None, 1)                 51        
                                                                 
Total params: 34,001
Trainable params: 34,001
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Adding checkpoints
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [27]:
# Saving checkpoints
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804
    )

In [28]:
# Compiling model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5683 - accuracy: 0.7229
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7327
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accuracy: 0.7327
Epoch 5/100
777/804 [===========================>..] - ETA: 0s - loss: 0.5489 - accuracy: 0.7340
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7338
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7352
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5464 - accuracy: 0.7360
Epoch 9/100
804/804 [=============================

804/804 [==============================] - 1s 1ms/step - loss: 0.5339 - accuracy: 0.7418
Epoch 65/100
770/804 [===========================>..] - ETA: 0s - loss: 0.5341 - accuracy: 0.7403
Epoch 00065: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5337 - accuracy: 0.7409
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5333 - accuracy: 0.7409
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5336 - accuracy: 0.7418
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5335 - accuracy: 0.7407
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5333 - accuracy: 0.7417
Epoch 70/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5331 - accuracy: 0.7415
Epoch 00070: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5333 - accuracy: 0.7415
Epoch 71/100
80

In [31]:
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5695 - accuracy: 0.7257 - 209ms/epoch - 781us/step
Loss: 0.5694645047187805, Accuracy: 0.7257142663002014


### Optimzation 3: Adding Epochs

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 200)               8800      
                                                                 
 dense_12 (Dense)            (None, 100)               20100     
                                                                 
 dense_13 (Dense)            (None, 50)                5050      
                                                                 
 dense_14 (Dense)            (None, 1)                 51        
                                                                 
Total params: 34,001
Trainable params: 34,001
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Adding checkpoints
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [35]:
# Saving checkpoints
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*804
    )

In [36]:
# Compiling the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200, callbacks=[cp_callback])
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
804/804 [==============================] - 2s 1ms/step - loss: 0.5673 - accuracy: 0.7255
Epoch 2/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7310
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7335
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7333
Epoch 5/200
789/804 [============================>.] - ETA: 0s - loss: 0.5493 - accuracy: 0.7340
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7338
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7352
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7347
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7356
Epoch 9/200
804/804 [=============================

804/804 [==============================] - 1s 2ms/step - loss: 0.5347 - accuracy: 0.7409
Epoch 65/200
797/804 [============================>.] - ETA: 0s - loss: 0.5331 - accuracy: 0.7426
Epoch 00065: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5334 - accuracy: 0.7422
Epoch 66/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5333 - accuracy: 0.7427
Epoch 67/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5333 - accuracy: 0.7420
Epoch 68/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5338 - accuracy: 0.7414
Epoch 69/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5340 - accuracy: 0.7419
Epoch 70/200
795/804 [============================>.] - ETA: 0s - loss: 0.5334 - accuracy: 0.7419
Epoch 00070: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5333 - accuracy: 0.7420
Epoch 71/200
80

Epoch 126/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5342 - accuracy: 0.7423
Epoch 127/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5318 - accuracy: 0.7421
Epoch 128/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5310 - accuracy: 0.7421
Epoch 129/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5312 - accuracy: 0.7432
Epoch 130/200
803/804 [============================>.] - ETA: 0s - loss: 0.5312 - accuracy: 0.7431
Epoch 00130: saving model to checkpoints/weights.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5311 - accuracy: 0.7430
Epoch 131/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5313 - accuracy: 0.7426
Epoch 132/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5310 - accuracy: 0.7423
Epoch 133/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5315 - accuracy: 0.7423
Epoch 134/200
804/804 [==========

804/804 [==============================] - 1s 1ms/step - loss: 0.5300 - accuracy: 0.7426
Epoch 189/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5301 - accuracy: 0.7435
Epoch 190/200
775/804 [===========================>..] - ETA: 0s - loss: 0.5302 - accuracy: 0.7425
Epoch 00190: saving model to checkpoints/weights.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5301 - accuracy: 0.7425
Epoch 191/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5299 - accuracy: 0.7421
Epoch 192/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5303 - accuracy: 0.7427
Epoch 193/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5302 - accuracy: 0.7432
Epoch 194/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5300 - accuracy: 0.7428
Epoch 195/200
774/804 [===========================>..] - ETA: 0s - loss: 0.5287 - accuracy: 0.7450
Epoch 00195: saving model to checkpoints/weights.19

In [38]:
# Evaluate the model
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5695 - accuracy: 0.7257 - 222ms/epoch - 827us/step
Loss: 0.5694645047187805, Accuracy: 0.7257142663002014


In [39]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_Optimzation.h5")